In [1]:
from keras.layers import Embedding, CuDNNLSTM, Bidirectional, Dense, CuDNNGRU
from keras.initializers import Constant
from keras import Sequential
import keras.backend as K
import numpy as np
from keras.backend import epsilon
import ipdb

EMBEDDING_DIM = 300
INPUT_LENGTH = 50


class Model:
    def __init__(self, tokenizer, embedding_matrix,
                 rnn_units=50,
                 bidirectional=True,
                 rnn_type='lstm',
                 show_summary=True):
        rnn_types = {
            'lstm': CuDNNLSTM,
            'gru': CuDNNGRU
        }
        rnn_type = rnn_types[rnn_type]

        # load pre-trained word embeddings into an Embedding layer
        # note that we set trainable = False so as to keep the embeddings fixed
        num_words = len(tokenizer.word_index) + 1
        embedding_layer = Embedding(num_words,
                                    EMBEDDING_DIM,
                                    embeddings_initializer=Constant(embedding_matrix),
                                    input_length=INPUT_LENGTH,
                                    trainable=False)
        model = Sequential()
        model.add(embedding_layer)
        if bidirectional:
            model.add(Bidirectional(rnn_type(rnn_units)))
        else:
            model.add(rnn_type(rnn_units))
        model.add(Dense(num_words, activation='softmax'))
        if show_summary:
            model.summary()

        self.model = model
        self.tokenizer = tokenizer

    def train(self, X, y, epochs=5, batch_size=32, callbacks=[]):
        model = self.model
        # compile network
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[_perplexity])
        # fit network
        model.fit(X, y, 
                  epochs=epochs,
                  batch_size=batch_size,
                  verbose=1, 
                  shuffle=True,
                  validation_split=0.2,
                  callbacks=callbacks)

    def predict(self, first_word, n_words, B=3):
        tokenizer = self.tokenizer
        model = self.model
        in_text, result = first_word, [first_word]
        encoded = get_encoded(in_text, tokenizer)
        beam_sequences_scores = [[encoded, 0]]

        while len(result) < n_words:
            all_candidates = []
            beam_sequences_scores = self.beam_step(beam_sequences_scores, B)
            for seq_score in beam_sequences_scores:
                seq_scores = self.beam_step([seq_score], B)
                all_candidates.append(seq_scores)
            flatten = lambda lst: [item for sublist in lst for item in sublist]
            beam_sequences_scores = sorted(flatten(all_candidates), reverse=True, key=lambda tup: tup[1])[:B]
            result, _= beam_sequences_scores[0]
        
        words = [get_word(token, self.tokenizer) for token in result]
        return ' '.join(words)
    
    
    def beam_step(self, beam_sequences_scores, B):            
        all_candidates = []
        for seq, score in beam_sequences_scores: # for each sequence
            # predict top B words
            seq_pad = pad_sequences([[sample] for sample in seq], maxlen=INPUT_LENGTH)
            words_probs = self.model.predict_proba(seq_pad, verbose=0)[0]
            words_probs_enu = list(enumerate(words_probs))
            words_probs_sorted = sorted(words_probs_enu, key=lambda x: x[1], reverse=True) # sorting in descending order
            top_b_words_probs = words_probs_sorted[:B] # top B words with max probability
            # for each prob in top B words, create a candidate
            for token, prob in top_b_words_probs: 
                word_token = token
                candidate = [np.append(seq, word_token), score + np.log(prob + epsilon())] # todo: word_token 
                all_candidates.append(candidate)
        # take candidates with max score
        beam_sequences_scores = sorted(all_candidates, reverse=True, key=lambda tup: tup[1])[:B]
        return beam_sequences_scores

def get_encoded(text, tokenizer):
    encoded = tokenizer.texts_to_sequences([text])[0]
    encoded = np.array(encoded)
    return encoded

def get_word(index, tokenizer):
    for word, idx in tokenizer.word_index.items():
         if idx == index:
            return word

def _perplexity(y_true, y_pred):
    cross_entropy = K.categorical_crossentropy(y_true, y_pred)
    perplexity = K.pow(2.0, cross_entropy)
    return perplexity


Using TensorFlow backend.


In [2]:
import warnings
warnings.filterwarnings("ignore")
import src.dataset as ds
import numpy as np
from src.embeddings import extract_embedding_weights
from keras.layers import Embedding, CuDNNLSTM, Bidirectional, Dense, CuDNNGRU
from keras.initializers import Constant
from keras import Sequential
from tensorflow.python.keras.callbacks import TensorBoard
# from src.model import Model

from time import time
import tensorflow as tf


In [3]:
X, y, tokenizer = ds.load_tokenized_data()
embedding_matrix = extract_embedding_weights()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 615/615 [00:01<00:00, 363.80it/s]


In [4]:
from keras.preprocessing.sequence import pad_sequences
X = pad_sequences([[sample] for sample in X], maxlen=INPUT_LENGTH)

In [5]:
model = Model(tokenizer, embedding_matrix, rnn_units=150, rnn_type='gru')

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 300)           2251800   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300)               406800    
_________________________________________________________________
dense_1 (Dense)              (None, 7506)              2259306   
Total params: 4,917,906
Trainable params: 2,666,106
Non-trainable params: 2,251,800
_________________________________________________________________


In [6]:
model.train(X,y, epochs=10, batch_size=20)

Instructions for updating:
Use tf.cast instead.
Train on 154596 samples, validate on 38650 samples
Epoch 1/10
154596/154596 [==============================] - 90s 581us/step - loss: 4.9001 - _perplexity: 867.2607 - val_loss: 5.3807 - val__perplexity: 2141.3858
Epoch 2/10
154596/154596 [==============================] - 82s 529us/step - loss: 4.3700 - _perplexity: 493.4785 - val_loss: 5.4135 - val__perplexity: 3428.7785
Epoch 3/10
154596/154596 [==============================] - 83s 540us/step - loss: 4.2017 - _perplexity: 401.3853 - val_loss: 5.4521 - val__perplexity: 4138.1159
Epoch 4/10
154596/154596 [==============================] - 84s 546us/step - loss: 4.1103 - _perplexity: 374.2492 - val_loss: 5.4633 - val__perplexity: 4357.1526
Epoch 5/10
154596/154596 [==============================] - 83s 540us/step - loss: 4.0496 - _perplexity: 364.8819 - val_loss: 5.5017 - val__perplexity: 4483.3748
Epoch 6/10
154596/154596 [==============================] - 82s 533us/step - loss: 4.0055 -

In [7]:
model.predict('Deep', 9)

'deep within within within within within within within within'